# Synthetic Data Study: Samoa Model
The aim of this is to determine what quality of data is required for a successful run of the pypei method.

In [762]:
import numpy as np
import pypei
import casadi as ca

from scipy import integrate, interpolate, optimize
from matplotlib import pyplot as plt
from matplotlib import font_manager

In [763]:
label_font = font_manager.FontProperties(
    family=['cmr10'],
    weight='regular',
    size=16,
)
legend_font = font_manager.FontProperties(
    family=['cmr10'],
    weight='regular',
    size=14,
)
tick_font = font_manager.FontProperties(
    family=['cmr10'],
    weight='regular',
    size=12,
)
plt.rcParams['figure.constrained_layout.use'] = True
plt.rcParams.update(
    {
        'text.usetex': False,
        'font.family': 'cmr10',
        'font.serif': 'cmr10',
        'mathtext.fontset': 'cm',
        "mathtext.default": 'regular',
    }
)

In [2]:
rng = np.random.default_rng()

In [3]:
%matplotlib widget
plt.rcParams['figure.constrained_layout.use'] = True

In [4]:
def seir_model(t, y, p):
    b, g, e, d, a, m, mH = p[:7]
    S,E,I,H,G,R,D,cE,cH = y[:9]
    return [
        -b*S*I/(S+E+I+R+G+H),
        b*S*I/(S+E+I+R+G+H)-g*E,
        g*E-(e+a+m)*I,
        e*I-(d+mH)*H,
        d*H,
        a*I,
        m*I + mH*H,
        g*E,
        e*I
    ]

# Truth Generation

In [12]:
# Solve the model forward in time with sensible parameters

true_parameters = [
    5,        # beta, transmission
    0.25,     # gamma, latent
    0.05,     # eta, hospitalisation
    0.2,      # delta, hospital recovery
    0.2,      # alpha, unhospitalised recovery
    0.005,    # mu, unhospitalised death
    0.015,    # muH, hospitalised death
]

initial_conditions = [
    20_000,
    5,
    5,
    0,
    0,
    180_000,
    0,
    0,
    0,
]

solve_time = [0, 240]

true_solution = integrate.solve_ivp(seir_model, solve_time, initial_conditions, args=[true_parameters], dense_output=True)

In [13]:
plot_times = np.linspace(*solve_time, 1001)

plt.figure()
plt.plot(plot_times, true_solution.sol(plot_times).T)
plt.legend('SEIHGRDeh')
plt.ylim([-initial_conditions[0]*0.1, initial_conditions[0]*1.1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-2000.0, 22000.0)

# Maximum quality data experiment

All states observed.
No observational noise.

In [14]:
# Generate max quality data: all states observed, no noise

data_time_max_qual = np.arange(200)
data_max_qual = true_solution.sol(data_time_max_qual)

In [15]:
model_form = {
    'state': 9,
    'parameters': 7,
}
model_config = {
    'grid_size': 200,
    'basis_number': 40,
    'model_form': model_form,
    'time_span': solve_time,
    'model': seir_model,
}
model = pypei.modeller.Model(model_config)

In [16]:
comparator = model.all_x_at(model.cs, data_time_max_qual).reshape((-1, 1))
data_max_qual_flat = data_max_qual.flatten()

In [17]:
data_L = pypei.objective.Objective._autoconfig_L(data_max_qual)
pypei.objective.L_via_data(data_L, data_max_qual)

In [18]:
mdl_L = {
    'n': np.prod(model.xs.shape),
    'iid': False,
    'balance': False,
    'numL': 5,
    'struct': pypei.objective.map_order_to_L_struct(['SCR', 'EI', 'H', 'GA', 'D'], model_config['grid_size'], 'SEIHGRDCA')
}

In [19]:
objective_config = {
    'Y': [
        {
            'sz': data_max_qual.shape,
            'obs_fn': comparator,
        },
        {
            'sz': model.xs.shape,
            'unitary': True,
            'obs_fn': pypei.objective.Objective._MODELFIT(model),
        },
    ],
    'L': [
        data_L,
        # pypei.objective.Objective._autoconfig_L(model.xs,),
        mdl_L,
    ]
}
objective = pypei.objective.Objective()
objective.make(objective_config)

In [20]:
solver = pypei.irls_fitter.Solver(objective=objective)
solver_config = solver.make_config(model, objective)
# add in a physical constraint to prevent degenerate solutions
# cE(0) > E(0) + I(0)
# solver_config['g'] = ca.vcat([solver_config['g'],
#                               model.xs[0,7] - model.xs[0,2],
#                               model.xs[0,8] - model.xs[0,4]])
solver_config['o'] = {'ipopt': {'max_iter': 5000, 'print_frequency_iter': 100}}
# solver_config['o'] = pypei.fitter.ipopt_silent
solver.make(solver_config)

In [21]:
x0 = solver.proto_x0(model)
x0x0 = x0['x0']
x0x0[:-(model_form['parameters'])] = np.random.poisson(x0x0[:-(model_form['parameters'])] * 10000)
x0x0[-(model_form['parameters']):] = x0x0[-(model_form['parameters']):] * 0.2

lbx = -np.inf*x0x0
lbx[-(model_form['parameters']):] = 0

ubx = np.inf*x0x0

In [22]:
def p(w, y_noisy):
    return [*w, *[i for l in y_noisy for i in l], 0, ]#0.2]

def gaussian_w(residual, n):
    return 1/np.sqrt(float(ca.sumsqr(residual))/n)

def struct_weight(residuals, structure):
    ws = []
    # deal with data weights
    for s in structure[0]:
        ws.append(gaussian_w(residuals[0][s['i0']:s['i0']+s['n']], s['n']))
    # deal with model weights
    for s in structure[1]:
        if 'i0s' in s:
            rs = ca.vcat([residuals[1][i0:i0+n] for n, i0 in zip(s['ns'], s['i0s'])])
            ws.append(gaussian_w(rs, sum(s['ns'])))
        else:
            ws.append(gaussian_w(residuals[1][s['i0']:s['i0']+s['n']], s['n']))
    return ws
weight_args = {'structure': (data_L['struct'], mdl_L['struct'])}

In [23]:
w0 = [1, 1, 1, 1, 1, 1, 1, 1, 1,
#       1e-4, 1e-4, 1e-4, 1e-4, 1e-4]
#       1e-3, 1e-3, 1e-3, 1e-3, 1e-3]
      2e-2, 2e-2, 2e-2, 2e-2, 2e-2]
#       5e-1, 5e-1, 5e-1, 5e-1, 5e-1]
#       1, 1, 1, 1, 1,]
#       1e-6, 1e-6, 1e-6, 1e-6, 1e-6]

In [24]:
sol, ws, shist, whist, rshist, errhist = solver.irls(x0x0, p, y=data_max_qual, nit=5, lbx=lbx, ubx=ubx,
                                    w0=w0, weight=struct_weight, weight_args=weight_args, 
                                    hist=True)

Iteration: 0

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:     6822
Number of nonzeros in Lagrangian Hessian.............:     7696

Total number of variables............................:      367
                     variables with only lower bounds:        7
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total n

/media/dwu402/Data/cashpack/pypei/irls_fitter.py:231: StepControlWarning: Log-Likelihood is not improving: -6.344744695428493e-09
  warnings.warn(f"Log-Likelihood is not improving: {err}", category=Solver.StepControlWarning)


In [27]:
plt.figure()
plt.plot(model.observation_times, solver.get_state(shist[-1], model))
plt.gca().set_prop_cycle(None)
plt.plot(data_time_max_qual, data_max_qual.T, 'x')
plt.ylim(0, 20000)
plt.legend(['S','E','I','H','G','R','D','cE','cH'])
plt.title("Fit to perfect data with IRLS-GP")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Fit to perfect data with IRLS-GP')

# Comparison to scipy.optimize (NLS method)
We will do an apples to apples comparison, using the same initial guess.

In [19]:
from scipy import optimize

def f(p):
    x0 = p[7:16]
    s = integrate.solve_ivp(seir_model, solve_time, x0, args=[p[:7]], dense_output=True)
    comp = s.sol(data_time_max_qual)
    return np.linalg.norm((comp - data_max_qual).flatten())**2
    
minres = optimize.minimize(f, [0.2]*7 + [10_000]*9, options={'disp': True})

print(minres.message)

         Current function value: 2129496785.191323
         Iterations: 194
         Function evaluations: 5878
         Gradient evaluations: 345
Desired error not necessarily achieved due to precision loss.


In [20]:
def explore_minres(minres):
    x0 = minres.x[7:16]
    s = integrate.solve_ivp(seir_model, solve_time, x0, args=[minres.x[:7]], dense_output=True)
    comp = s.sol(data_time_max_qual)
    plt.figure()
    plt.plot(data_time_max_qual, comp.T)
    plt.gca().set_prop_cycle(None)
    plt.plot(data_time_max_qual, data_max_qual.T, 'x')
    plt.ylim(0, 20_000)
    plt.xlim(None, 250)

explore_minres(minres)
plt.title("Fit to perfect data with NLS")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Fit to perfect data with NLS')

# Fit to partially observed data [Stage 1]
We start removing some states from the observed data.
In Stage 1 we will remove the exposed category.

In [28]:
stage_1_y0idxs = [0, 2, 3, 4, 5, 6, 7, 8]
data_po_stage_1 = data_max_qual[stage_1_y0idxs,:]

comparator_stage_1 = model.all_x_at(model.cs, data_time_max_qual)[:,stage_1_y0idxs].reshape((-1,1))
data_L_stage_1 = pypei.objective.Objective._autoconfig_L(data_po_stage_1)
pypei.objective.L_via_data(data_L_stage_1, data_po_stage_1)

In [29]:
objective_config_stage_1 = {
    'Y': [
        {
            'sz': data_po_stage_1.shape,
            'obs_fn': comparator_stage_1,
        },
        {
            'sz': model.xs.shape,
            'unitary': True,
            'obs_fn': pypei.objective.Objective._MODELFIT(model),
        },
    ],
    'L': [
        data_L_stage_1,
        # pypei.objective.Objective._autoconfig_L(model.xs,),
        mdl_L,
    ]
}
objective_stage_1 = pypei.objective.Objective()
objective_stage_1.make(objective_config_stage_1)

In [30]:
solver_stage_1 = pypei.irls_fitter.Solver(objective=objective_stage_1)
solver_config_stage_1 = solver_stage_1.make_config(model, objective_stage_1)
# add in a physical constraint to prevent degenerate solutions
# cE(0) > E(0) + I(0)
# solver_config['g'] = ca.vcat([solver_config['g'],
#                               model.xs[0,7] - model.xs[0,2],
#                               model.xs[0,8] - model.xs[0,4]])
solver_config_stage_1['o'] = {'ipopt': {'max_iter': 5000, 'print_frequency_iter': 100}}
# solver_config['o'] = pypei.fitter.ipopt_silent
solver_stage_1.make(solver_config_stage_1)

In [31]:
weight_args_stage_1 = {'structure': (data_L_stage_1['struct'], mdl_L['struct'])}
w0_s1 = [1.0]*8 + [1e-2]*5

In [32]:
solver_stage_1_output = solver_stage_1.irls(x0x0, p, y=data_po_stage_1, nit=5, lbx=lbx,
                                    w0=w0_s1, weight=struct_weight, weight_args=weight_args_stage_1, 
                                    hist=True)

sol_s1, ws_s1, shist_s1, whist_s1, rshist_s1, errhist_s1 = solver_stage_1_output

Iteration: 0
This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:     6822
Number of nonzeros in Lagrangian Hessian.............:     7696

Total number of variables............................:      367
                     variables with only lower bounds:        7
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:     1800
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_

/media/dwu402/Data/cashpack/pypei/irls_fitter.py:231: StepControlWarning: Log-Likelihood is not improving: -8.393810204875756e-09
  warnings.warn(f"Log-Likelihood is not improving: {err}", category=Solver.StepControlWarning)


In [33]:
plt.figure()
plt.plot(model.observation_times, solver_stage_1.get_state(shist_s1[-1], model))
plt.plot(data_time_max_qual, data_max_qual.T, 'x')
plt.ylim(0, 20000)
plt.legend(['S','E','I','H','G','R','D','cE','cH'])
plt.title("Fit to im-perfect data [stage1] with IRLS-GP")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Fit to im-perfect data [stage1] with IRLS-GP')

In [34]:
plt.figure()
plt.semilogy([np.squeeze(solver_stage_1.get_parameters(s, model)) for s in shist_s1])
plt.gca().set_prop_cycle(None)
for i, p_by_i in enumerate(true_parameters):
    plt.semilogy(range(len(shist_s1)), [p_by_i]*len(shist_s1), 'o', alpha=0.7, mfc='None')
plt.title("Parameters by iteration")
plt.legend("bgedamH")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [35]:
obj_parts = ca.Function("objparts", [solver_stage_1.decision_vars, solver_stage_1.parameters], [objective_stage_1.us_obj_fn(i) for i in range(2)])

obj_parts_real = []
for si, wi in zip(shist_s1, whist_s1):
    obj_parts_real.append(np.squeeze(obj_parts(si['x'], p(wi, data_po_stage_1))))
    
plt.figure()
plt.loglog(*zip(*obj_parts_real), 'o-')
plt.loglog(*obj_parts_real[0], 'rx')
plt.xlabel("Data Error")
plt.ylabel("Model Error")
plt.title("Error Tradeoff L Curve")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Error Tradeoff L Curve')

# Stage 2: No E, No I

In [36]:
stage_2_y0idxs = [0, 3, 4, 5, 6, 7, 8]
data_po_stage_2 = data_max_qual[stage_2_y0idxs,:]

comparator_stage_2 = model.all_x_at(model.cs, data_time_max_qual)[:,stage_2_y0idxs].reshape((-1,1))
data_L_stage_2 = pypei.objective.Objective._autoconfig_L(data_po_stage_2)
pypei.objective.L_via_data(data_L_stage_2, data_po_stage_2)

In [37]:
objective_config_stage_2 = {
    'Y': [
        {
            'sz': data_po_stage_2.shape,
            'obs_fn': comparator_stage_2,
        },
        {
            'sz': model.xs.shape,
            'unitary': True,
            'obs_fn': pypei.objective.Objective._MODELFIT(model),
        },
    ],
    'L': [
        data_L_stage_2,
        # pypei.objective.Objective._autoconfig_L(model.xs,),
        mdl_L,
    ]
}
objective_stage_2 = pypei.objective.Objective()
objective_stage_2.make(objective_config_stage_2)

In [38]:
solver_stage_2 = pypei.irls_fitter.Solver(objective=objective_stage_2)
solver_config_stage_2 = solver_stage_2.make_config(model, objective_stage_2)
# add in a physical constraint to prevent degenerate solutions
# cE(0) > E(0) + I(0)
# solver_config['g'] = ca.vcat([solver_config['g'],
#                               model.xs[0,7] - model.xs[0,2],
#                               model.xs[0,8] - model.xs[0,4]])
solver_config_stage_2['o'] = {'ipopt': {'max_iter': 5000, 'print_frequency_iter': 100}}
# solver_config['o'] = pypei.fitter.ipopt_silent
solver_stage_2.make(solver_config_stage_2)

In [39]:
weight_args_stage_2 = {'structure': (data_L_stage_2['struct'], mdl_L['struct'])}
w0_s2 = [1.0]*7 + [1e-2]*5

In [40]:
solver_stage_2_output = solver_stage_2.irls(x0x0, p, y=data_po_stage_2, nit=5, lbx=lbx,
                                    w0=w0_s2, weight=struct_weight, weight_args=weight_args_stage_2, 
                                    hist=True)

sol_s2, ws_s2, shist_s2, whist_s2, rshist_s2, errhist_s2 = solver_stage_2_output

Iteration: 0
This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:     6822
Number of nonzeros in Lagrangian Hessian.............:     7696

Total number of variables............................:      367
                     variables with only lower bounds:        7
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:     1800
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_

/media/dwu402/Data/cashpack/pypei/irls_fitter.py:231: StepControlWarning: Log-Likelihood is not improving: -1.6407787250310813e-10
  warnings.warn(f"Log-Likelihood is not improving: {err}", category=Solver.StepControlWarning)


In [41]:
plt.figure()
plt.plot(model.observation_times, solver_stage_2.get_state(shist_s2[-1], model))
plt.plot(data_time_max_qual, data_max_qual.T, 'x')
plt.ylim(0, 20000)
plt.legend(['S','E','I','H','G','R','D','cE','cH'])
plt.title("Fit to im-perfect data [stage2] with IRLS-GP")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Fit to im-perfect data [stage2] with IRLS-GP')

## Stage 3: No E, I, R

In [42]:
stage_3_y0idxs = [0, 3, 4, 6, 7, 8]
data_po_stage_3 = data_max_qual[stage_3_y0idxs,:]

comparator_stage_3 = model.all_x_at(model.cs, data_time_max_qual)[:,stage_3_y0idxs].reshape((-1,1))
data_L_stage_3 = pypei.objective.Objective._autoconfig_L(data_po_stage_3)
pypei.objective.L_via_data(data_L_stage_3, data_po_stage_3)

In [43]:
objective_config_stage_3 = {
    'Y': [
        {
            'sz': data_po_stage_3.shape,
            'obs_fn': comparator_stage_3,
        },
        {
            'sz': model.xs.shape,
            'unitary': True,
            'obs_fn': pypei.objective.Objective._MODELFIT(model),
        },
    ],
    'L': [
        data_L_stage_3,
        # pypei.objective.Objective._autoconfig_L(model.xs,),
        mdl_L,
    ]
}
objective_stage_3 = pypei.objective.Objective()
objective_stage_3.make(objective_config_stage_3)

In [44]:
solver_stage_3 = pypei.irls_fitter.Solver(objective=objective_stage_3)
solver_config_stage_3 = solver_stage_3.make_config(model, objective_stage_3)
solver_config_stage_3['o'] = {'ipopt': {'max_iter': 5000, 'print_frequency_iter': 100}}
solver_stage_3.make(solver_config_stage_3)

In [45]:
weight_args_stage_3 = {'structure': (data_L_stage_3['struct'], mdl_L['struct'])}
w0_s3 = [1.0]*6 + [1e-2]*5

In [46]:
solver_stage_3_output = solver_stage_3.irls(x0x0, p, y=data_po_stage_3, nit=5, lbx=lbx,
                                    w0=w0_s3, weight=struct_weight, weight_args=weight_args_stage_3, 
                                    hist=True)

sol_s3, ws_s3, shist_s3, whist_s3, rshist_s3, errhist_s3 = solver_stage_3_output

Iteration: 0
This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:     6822
Number of nonzeros in Lagrangian Hessian.............:     7696

Total number of variables............................:      367
                     variables with only lower bounds:        7
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:     1800
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_

/media/dwu402/Data/cashpack/pypei/irls_fitter.py:231: StepControlWarning: Log-Likelihood is not improving: -5.934651461979463e-09
  warnings.warn(f"Log-Likelihood is not improving: {err}", category=Solver.StepControlWarning)


In [47]:
plt.figure()
plt.plot(model.observation_times, solver_stage_3.get_state(shist_s3[-1], model))
plt.gca().set_prop_cycle(None)

plt.plot(data_time_max_qual, data_max_qual.T, 'x')
plt.ylim(0, 20000)
plt.legend(['S','E','I','H','G','R','D','cE','cH'])
plt.title("Fit to im-perfect data [stage3] with IRLS-GP")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Fit to im-perfect data [stage3] with IRLS-GP')

## Stage 4: Samoa-like Data; no S, E, I, R

In [48]:
stage_4_y0idxs = [3, 4, 6, 7, 8]
data_po_stage_4 = data_max_qual[stage_4_y0idxs,:]

comparator_stage_4 = model.all_x_at(model.cs, data_time_max_qual)[:,stage_4_y0idxs].reshape((-1,1))
data_L_stage_4 = pypei.objective.Objective._autoconfig_L(data_po_stage_4)
pypei.objective.L_via_data(data_L_stage_4, data_po_stage_4)

In [49]:
objective_config_stage_4 = {
    'Y': [
        {
            'sz': data_po_stage_4.shape,
            'obs_fn': comparator_stage_4,
        },
        {
            'sz': model.xs.shape,
            'unitary': True,
            'obs_fn': pypei.objective.Objective._MODELFIT(model),
        },
    ],
    'L': [
        data_L_stage_4,
        # pypei.objective.Objective._autoconfig_L(model.xs,),
        mdl_L,
    ]
}
objective_stage_4 = pypei.objective.Objective()
objective_stage_4.make(objective_config_stage_4)

In [50]:
solver_stage_4 = pypei.irls_fitter.Solver(objective=objective_stage_4)
solver_config_stage_4 = solver_stage_4.make_config(model, objective_stage_4)
solver_config_stage_4['o'] = {'ipopt': {'max_iter': 5000, 'print_frequency_iter': 100}}
solver_stage_4.make(solver_config_stage_4)

In [51]:
weight_args_stage_4 = {'structure': (data_L_stage_4['struct'], mdl_L['struct'])}
w0_s4 = [1.0]*5 + [1e-2]*5

In [52]:
solver_stage_4_output = solver_stage_4.irls(x0x0, p, y=data_po_stage_4, nit=5, lbx=lbx, lbg=0,
                                    w0=w0_s4, weight=struct_weight, weight_args=weight_args_stage_4, 
                                    hist=True)

sol_s4, ws_s4, shist_s4, whist_s4, rshist_s4, errhist_s4 = solver_stage_4_output

Iteration: 0
This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:     6822
Number of nonzeros in Lagrangian Hessian.............:     7696

Total number of variables............................:      367
                     variables with only lower bounds:        7
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:     1800
        inequality constraints with only lower bounds:     1800
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_

In [53]:
plt.figure()
plt.plot(model.observation_times, solver_stage_4.get_state(shist_s4[2], model))
plt.gca().set_prop_cycle(None)
plt.plot(data_time_max_qual, data_max_qual.T, 'x')
# plt.ylim(0, 20000)
plt.legend(['S','E','I','H','G','R','D','cE','cH'])
plt.title("Fit to noise-free Samoa-like data [stage 4] with IRLS-GP")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Fit to noise-free Samoa-like data [stage 4] with IRLS-GP')

In [54]:
plt.figure()
plt.semilogy([np.squeeze(solver.get_parameters(s, model)) for s in shist_s4])
plt.gca().set_prop_cycle(None)
for i, p_by_i in enumerate(true_parameters):
    plt.semilogy(range(len(shist_s4)), [p_by_i]*len(shist_s4), 'o', alpha=0.7, mfc='None')
plt.title("Parameters by iteration")
plt.legend(list("bgedam") + ['muH', 'Truth'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [55]:
obj_parts_stage_4 = ca.Function("objparts_s4", [solver_stage_4.decision_vars, solver_stage_4.parameters], [objective_stage_4.us_obj_fn(i) for i in range(2)])

obj_parts_real_stage_4 = []
for si, wi in zip(shist_s4, whist_s4):
    obj_parts_real_stage_4.append(np.squeeze(obj_parts_stage_4(si['x'], p(wi, data_po_stage_4))))
    
plt.figure()
plt.loglog(*zip(*obj_parts_real_stage_4), 'o-')
plt.loglog(*obj_parts_real_stage_4[0], 'rx')
plt.xlabel("Data Error")
plt.ylabel("Model Error")
plt.title("Error Tradeoff L Curve [Stage 4]")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Error Tradeoff L Curve [Stage 4]')

## Stage 4a: Samoa-like Data; no S, E, I, R; give S(0)+R(0)?

In [56]:
objective_config_stage_4a = {
    'Y': [
        {
            'sz': data_po_stage_4.shape,
            'obs_fn': comparator_stage_4,
        },
        {
            'sz': model.xs.shape,
            'unitary': True,
            'obs_fn': pypei.objective.Objective._MODELFIT(model),
        },
        {
            'sz': (1, 1),
            'unitary': True,
            'obs_fn': ca.sum2(model.all_x_at(model.cs, 0)[[0,5]]),
        }
    ],
    'L': [
        data_L_stage_4,
        # pypei.objective.Objective._autoconfig_L(model.xs,),
        mdl_L,
        {
            'n': 1,
            'iid': True,
            'balance': False,
        },
    ]
}
objective_stage_4a = pypei.objective.Objective()
objective_stage_4a.make(objective_config_stage_4a)

In [57]:
solver_stage_4a = pypei.irls_fitter.Solver(objective=objective_stage_4a)
solver_config_stage_4a = solver_stage_4a.make_config(model, objective_stage_4a)
solver_config_stage_4a['o'] = {'ipopt': {'max_iter': 5000, 'print_frequency_iter': 100}}
solver_stage_4a.make(solver_config_stage_4a)

In [58]:
def p_4a(w, y_noisy):
    return [*w, *[i for l in y_noisy for i in l], 0, initial_conditions[0] + initial_conditions[5]]#0.2]

def struct_weight_4a(residuals, structure):
    ws = []
    # deal with data weights
    for s in structure[0]:
        ws.append(gaussian_w(residuals[0][s['i0']:s['i0']+s['n']], s['n']))
    # deal with model weights
    for s in structure[1]:
        if 'i0s' in s:
            rs = ca.vcat([residuals[1][i0:i0+n] for n, i0 in zip(s['ns'], s['i0s'])])
            ws.append(gaussian_w(rs, sum(s['ns'])))
        else:
            ws.append(gaussian_w(residuals[1][s['i0']:s['i0']+s['n']], s['n']))
    # addntl info
    ws.append(1.0)
#     ai_gw = gaussian_w(residuals[2], 1)
#     if ai_gw > 1e8:
# #         print("gweilo")
#         ws.append(1e8)
#     else:
#         ws.append(ai_gw)
    return ws


In [59]:
weight_args_stage_4a = {'structure': (data_L_stage_4['struct'], mdl_L['struct'])}
w0_s4a = [1.0]*5 + [1e-2]*5 + [1.0]

In [60]:
solver_stage_4a_output = solver_stage_4a.irls(x0x0, p_4a, y=data_po_stage_4, nit=5, lbx=lbx, lbg=0,
                                    w0=w0_s4a, weight=struct_weight_4a, weight_args=weight_args_stage_4a, 
                                    hist=True)

sol_s4a, ws_s4a, shist_s4a, whist_s4a, rshist_s4a, errhist_s4a = solver_stage_4a_output

Iteration: 0
This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:     6822
Number of nonzeros in Lagrangian Hessian.............:     7696

Total number of variables............................:      367
                     variables with only lower bounds:        7
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:     1800
        inequality constraints with only lower bounds:     1800
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_

In [61]:
plt.figure()
plt.plot(model.observation_times, solver_stage_4a.get_state(shist_s4a[-1], model))
plt.gca().set_prop_cycle(None)
plt.plot(data_time_max_qual, data_max_qual.T, 'x')
# plt.ylim(0, 20000)
plt.legend(['S','E','I','H','G','R','D','cE','cH'])
plt.title("Fit to noise-free Samoa-like data (with N(0)) [stage 4a] with IRLS-GP")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Fit to noise-free Samoa-like data (with N(0)) [stage 4a] with IRLS-GP')

In [62]:
plt.figure()
plt.semilogy([np.squeeze(solver.get_parameters(s, model)) for s in shist_s4a])
plt.gca().set_prop_cycle(None)
for i, p_by_i in enumerate(true_parameters):
    plt.semilogy(range(len(shist_s4a)), [p_by_i]*len(shist_s4a), 'o', alpha=0.7, mfc='None')
plt.title("Parameters by iteration")
plt.legend(list("bgedam") + ['muH', 'Truth'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [63]:
plt.figure()
plt.semilogy(whist_s4a)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [64]:
obj_parts_stage_4a = ca.Function("objparts_s4a", [solver_stage_4a.decision_vars, solver_stage_4a.parameters], [objective_stage_4a.us_obj_fn(i) for i in range(2)])

obj_parts_real_stage_4a = []
for si, wi in zip(shist_s4a, whist_s4a):
    obj_parts_real_stage_4a.append(np.squeeze(obj_parts_stage_4a(si['x'], p_4a(wi, data_po_stage_4))))
    
plt.figure()
plt.loglog(*zip(*obj_parts_real_stage_4a), 'o-')
plt.loglog(*obj_parts_real_stage_4a[0], 'rx')
plt.xlabel("Data Error")
plt.ylabel("Model Error")
plt.title("Error Tradeoff L Curve [Stage 4a]")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Error Tradeoff L Curve [Stage 4a]')

## Stage 5 - Adding noise

Observational noise generated with a Gaussian additive error observation model

y ~ x + Normal(0, sigma)

In [67]:
stage_5_y0idxs = stage_4_y0idxs
noise_level = 0.3
noise_sigma = [noise_level * d.mean() for d in data_po_stage_4]

data_noisy_stage_5 = data_max_qual[stage_5_y0idxs,:]
for sigma, layer in zip(noise_sigma, data_L_stage_4['struct']):
#     data_noisy_stage_5[layer['i0']:layer['i0']+layer['n'], layer['i0']:layer['i0']+layer['n']] = rng.normal(
#         loc=data_noisy_stage_5[layer['i0']:layer['i0']+layer['n'], layer['i0']:layer['i0']+layer['n']], scale=sigma)
    data_noisy_stage_5[layer['i0']:layer['i0']+layer['n'], layer['i0']:layer['i0']+layer['n']] = rng.poisson(data_noisy_stage_5[layer['i0']:layer['i0']+layer['n'], layer['i0']:layer['i0']+layer['n']])

In [68]:
plt.figure()
plt.plot(data_time_max_qual, data_noisy_stage_5.T, 'o', mfc='none')
plt.plot(data_time_max_qual, data_po_stage_4.T);

/home/dwu402/.virtualenvs/scider/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [75]:
w0_s5 = [1.0]*5 + [0.75]*5 + [1.0]

In [76]:
solver_stage_5_output = solver_stage_4a.irls(x0x0, p_4a, y=data_noisy_stage_5, nit=5, lbx=lbx, lbg=0,
                                    w0=w0_s5, weight=struct_weight_4a, weight_args=weight_args_stage_4a, 
                                    hist=True)

sol_s5, ws_s5, shist_s5, whist_s5, rshist_s5, errhist_s5 = solver_stage_5_output

Iteration: 0
This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:     6822
Number of nonzeros in Lagrangian Hessian.............:     7696

Total number of variables............................:      367
                     variables with only lower bounds:        7
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:     1800
        inequality constraints with only lower bounds:     1800
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_

/media/dwu402/Data/cashpack/pypei/irls_fitter.py:231: StepControlWarning: Log-Likelihood is not improving: 0.934279846171119
  warnings.warn(f"Log-Likelihood is not improving: {err}", category=Solver.StepControlWarning)


 200  4.3564832e+05 0.00e+00 8.15e+04  -1.0 2.92e+04    -  1.00e+00 3.12e-02f  6
 300  4.4801708e+05 0.00e+00 3.73e+02  -1.0 8.84e+01  -9.3 1.00e+00 1.00e+00f  1
 400  2.1364312e+03 0.00e+00 3.73e+03  -3.8 3.05e+04    -  6.20e-01 6.45e-03f  8
 500  2.0951943e+03 0.00e+00 1.05e+01  -3.8 4.42e+01 -10.7 1.00e+00 1.00e+00h  1
 600  2.0650724e+03 0.00e+00 8.31e+00  -3.8 3.52e+01 -10.7 1.00e+00 1.00e+00h  1
 700  2.0412881e+03 0.00e+00 1.07e+03  -3.8 1.24e+04    -  1.00e+00 1.56e-02f  7
 800  2.0229185e+03 0.00e+00 1.57e+03  -3.8 1.64e+02 -12.1 1.00e+00 1.00e+00h  1
 900  2.0121296e+03 0.00e+00 4.30e+01  -3.8 1.50e+03    -  1.00e+00 6.25e-02h  5
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1000  1.8644997e+03 0.00e+00 6.08e+02  -5.7 1.77e+02 -11.7 1.00e+00 1.00e+00h  1
1100  1.8565511e+03 0.00e+00 1.58e+03  -5.7 2.16e+02 -12.0 1.00e+00 5.00e-01h  2
1200  1.8523856e+03 0.00e+00 9.22e+01  -5.7 2.10e+01 -10.9 1.00e+00 1.00e+00h  1
1300  1.8504225e+03 0.00e+00

/media/dwu402/Data/cashpack/pypei/irls_fitter.py:231: StepControlWarning: Log-Likelihood is not improving: 0.9300388077441605
  warnings.warn(f"Log-Likelihood is not improving: {err}", category=Solver.StepControlWarning)


In [77]:
plt.figure()
plt.semilogy([np.squeeze(solver.get_parameters(s, model)) for s in shist_s5])
plt.gca().set_prop_cycle(None)
for i, p_by_i in enumerate(true_parameters):
    plt.semilogy(range(len(shist_s5)), [p_by_i]*len(shist_s5), 'o', alpha=0.7, mfc='None')
plt.title("Parameters by iteration")
plt.legend(list("bgedam") + ['muH', 'Truth'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [82]:
plt.figure()
plt.plot(model.observation_times, solver_stage_4a.get_state(shist_s5[1], model))
plt.gca().set_prop_cycle(None)
plt.plot(data_time_max_qual, data_max_qual.T, 'o', mfc='none')
# plt.ylim(0, 30000)
plt.legend(['S','E','I','H','G','R','D','cE','cH'])
plt.title("Fit to im-perfect, noisy data [stage 5] with IRLS-GP")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Fit to im-perfect, noisy data [stage 5] with IRLS-GP')

In [79]:
plt.figure()
plt.semilogy(whist_s5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Stage 5a: Huber loss

In [84]:
objective_config_stage_5a = {
    'Y': [
        {
            'sz': data_po_stage_4.shape,
            'obs_fn': comparator_stage_4,
        },
        {
            'sz': model.xs.shape,
            'unitary': True,
            'obs_fn': pypei.objective.Objective._MODELFIT(model),
        },
    ],
    'L': [
        data_L_stage_4,
        mdl_L,
    ]
}
objective_stage_5a = pypei.objective.Objective()
objective_stage_5a.make(objective_config_stage_5a)

In [85]:
solver_stage_5a = pypei.irls_fitter.Solver(objective=objective_stage_5a)
solver_config_stage_5a = solver_stage_5a.make_config(model, objective_stage_5a)
solver_config_stage_5a['o'] = {'ipopt': {'max_iter': 5000, 'print_frequency_iter': 100}}
solver_stage_5a.make(solver_config_stage_5a)

In [90]:
def p_5a(w, y_noisy):
    return [*w, *[i for l in y_noisy for i in l], 0]

def struct_weight_5a(residuals, structure):
    ws = []
    # deal with data weights
    for s in structure[0]:
        ws.append(gaussian_w(residuals[0][s['i0']:s['i0']+s['n']], s['n']))
    # deal with model weights
    for s in structure[1]:
        if 'i0s' in s:
            rs = ca.vcat([residuals[1][i0:i0+n] for n, i0 in zip(s['ns'], s['i0s'])])
            ws.append(gaussian_w(rs, sum(s['ns'])))
        else:
            ws.append(gaussian_w(residuals[1][s['i0']:s['i0']+s['n']], s['n']))
    return ws

def huber_5a(residuals, structure, bounds):
    proposal_weights = struct_weight_5a(residuals, structure)
    lbnds, ubnds = zip(*bounds)
    weights = [float(np.clip(w, lbnd, ubnd)) for w, lbnd, ubnd in zip(proposal_weights, lbnds, ubnds)]
    return np.array(weights)

weight_args_stage_5a = {'structure': (data_L_stage_4['struct'], mdl_L['struct']), 'bounds': [[1e-2, None]]*5 + [[None, 1e2]]*5}
w0_s5a = [1.0]*5 + [1e-1]*5 


In [91]:
solver_stage_5a_output = solver_stage_5a.irls(x0x0, p_5a, y=data_noisy_stage_5, nit=5, lbx=lbx, lbg=0,
                                    w0=w0_s5a, weight=huber_5a, weight_args=weight_args_stage_5a, 
                                    hist=True)

sol_s5a, ws_s5a, shist_s5a, whist_s5a, rshist_s5a, errhist_s5a = solver_stage_5a_output

Iteration: 0
This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:     6822
Number of nonzeros in Lagrangian Hessian.............:     7696

Total number of variables............................:      367
                     variables with only lower bounds:        7
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:     1800
        inequality constraints with only lower bounds:     1800
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_

/media/dwu402/Data/cashpack/pypei/irls_fitter.py:231: StepControlWarning: Log-Likelihood is not improving: 0.9870590917830568
  warnings.warn(f"Log-Likelihood is not improving: {err}", category=Solver.StepControlWarning)


In [92]:
plt.figure()
plt.semilogy(whist_s5a)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [97]:
plt.figure()
plt.plot(model.observation_times, solver_stage_5a.get_state(shist_s5a[-2], model))
plt.gca().set_prop_cycle(None)
plt.plot(data_time_max_qual, data_max_qual.T, '.', mfc='none')
# plt.ylim(0, 30000)
plt.legend(['S','E','I','H','G','R','D','cE','cH'])
plt.title("Fit to im-perfect, noisy data [stage 5a] with IRLS-GP")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Fit to im-perfect, noisy data [stage 5a] with IRLS-GP')

In [98]:
plt.figure()
plt.semilogy([np.squeeze(solver.get_parameters(s, model)) for s in shist_s5a])
plt.gca().set_prop_cycle(None)
for i, p_by_i in enumerate(true_parameters):
    plt.semilogy(range(len(shist_s5a)), [p_by_i]*len(shist_s5a), 'o', alpha=0.7, mfc='None')
plt.title("Parameters by iteration")
plt.legend(list("bgedam") + ['muH', 'Truth'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# 5b: partial time window

In [743]:
data_time_partial_window = data_time_max_qual[:int(len(data_time_max_qual)//3.5):5]
data_5b = data_noisy_stage_5[:,:int(len(data_time_max_qual)//3.5):5]
comparator5b = model.all_x_at(model.cs, data_time_partial_window)[:,stage_4_y0idxs].reshape((-1,1))

In [744]:
plt.figure()
plt.plot(data_time_partial_window, data_5b.T)
plt.plot(data_time_max_qual, data_max_qual.T, '--')
plt.ylim(0, 25_000)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.0, 25000.0)

In [745]:
data_L_5b = pypei.objective.Objective._autoconfig_L(data_5b)
pypei.objective.L_via_data(data_L_5b, data_5b)

In [746]:
objective_config_stage_5b = {
    'Y': [
        {
            'sz': data_5b.shape,
            'obs_fn': comparator5b,
        },
        {
            'sz': model.xs.shape,
            'unitary': True,
            'obs_fn': pypei.objective.Objective._MODELFIT(model),
        },
    ],
    'L': [
        data_L_5b,
        mdl_L,
    ]
}

In [747]:
objective_stage_5b = pypei.objective.Objective()
objective_stage_5b.make(objective_config_stage_5b)

In [748]:
solver_stage_5b = pypei.irls_fitter.Solver(objective=objective_stage_5b)
solver_config_stage_5b = solver_stage_5b.make_config(model, objective_stage_5b)
solver_config_stage_5b['o'] = {'ipopt': {'max_iter': 5000, 'print_frequency_iter': 100}}
solver_stage_5b.make(solver_config_stage_5b)

In [749]:
weight_args_stage_5b = {'structure': (data_L_5b['struct'], mdl_L['struct']), 'bounds': [[3e-1, None]]*5 + [[None, 2]]*5}

In [750]:
solver_stage_5b_output = solver_stage_5b.irls(x0x0, p_5a, y=data_5b, nit=10, lbx=lbx, lbg=0,
                                    w0=w0_s5a, weight=huber_5a, weight_args=weight_args_stage_5b, 
                                    hist=True)

sol_s5b, ws_s5b, shist_s5b, whist_s5b, rshist_s5b, errhist_s5b = solver_stage_5b_output

Iteration: 0
This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:     6822
Number of nonzeros in Lagrangian Hessian.............:     7696

Total number of variables............................:      367
                     variables with only lower bounds:        7
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:     1800
        inequality constraints with only lower bounds:     1800
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_

SystemError: <built-in function Function_call> returned a result with an error set

In [ ]:
plt.figure()
plt.semilogy(whist_s5b)

In [293]:
plt.figure()
plt.plot(model.observation_times, solver_stage_5b.get_state(shist_s5b[5], model))
plt.gca().set_prop_cycle(None)
plt.plot(data_time_max_qual, data_max_qual.T, '.', mfc='none')
# plt.ylim(0, 30000)
plt.legend(['S','E','I','H','G','R','D','cE','cH'])
plt.title("Fit to im-perfect, noisy partial data [stage 5b] with IRLS-GP")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Fit to im-perfect, noisy partial data [stage 5b] with IRLS-GP')

In [294]:
plt.figure()
plt.semilogy([np.squeeze(solver.get_parameters(s, model)) for s in shist_s5b])
plt.gca().set_prop_cycle(None)
for i, p_by_i in enumerate(true_parameters):
    plt.semilogy(range(len(shist_s5b)), [p_by_i]*len(shist_s5b), 'o', alpha=0.7, mfc='None')
plt.title("Parameters by iteration")
plt.legend(list("bgedam") + ['muH', 'Truth'])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [295]:
get_cs = ca.Function('get_cs', [solver.decision_vars], model.cs)
plt.figure()
plt.plot(data_time_max_qual, data_max_qual[-2,:].T, 'k--')

finvals = []
for sh5b in shist_s5b:
    compare = model.all_x_at(get_cs(sh5b['x']), data_time_max_qual)[:,-2]
    finvals.append(float(compare[-1,0]))
    plt.plot(data_time_max_qual, compare)
plt.legend(['true'] + list(range(len(shist_s5b))))

plt.plot(data_time_partial_window, data_5b.T, '.')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [296]:
sol_10, w_10 = solver_stage_5b.irls(sol_s5b['x'], p_5a, y=data_5b, nit=1, lbx=lbx, lbg=0,
                                    w0=w0_s5a, weight=huber_5a, weight_args=weight_args_stage_5b, 
                                    hist=False)

Iteration: 0
This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:     6822
Number of nonzeros in Lagrangian Hessian.............:     7696

Total number of variables............................:      367
                     variables with only lower bounds:        7
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:     1800
        inequality constraints with only lower bounds:     1800
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_

In [297]:
plt.figure()
plt.plot(data_time_max_qual, data_max_qual[-2,:].T, 'k--')

finvals = []
for sh5b in shist_s5b:
    compare = model.all_x_at(get_cs(sh5b['x']), data_time_max_qual)[:,-2]
    finvals.append(float(compare[-1,0]))
    plt.plot(data_time_max_qual, compare)

compare = model.all_x_at(get_cs(sol_10['x']), data_time_max_qual)[:,-2]
finvals.append(float(compare[-1,0]))
plt.plot(data_time_max_qual, compare)

plt.plot(data_time_partial_window, data_5b.T, '.')
plt.legend(['true'] + list(range(len(shist_s5b)+1)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [298]:
plt.figure()
plt.plot(finvals)
plt.axhline(data_max_qual[-2,-1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [299]:
[np.linalg.norm(model.all_x_at(get_cs(x['x']), data_time_max_qual).T - data_max_qual) for x in shist_s5b]

[3003496.418470118,
 3242984.0872172075,
 3280520.1567298598,
 3307422.2281149426,
 3472382.3953973856,
 3401176.2707675393,
 3172930.9364247536,
 3657621.314735094,
 3357048.7838697024,
 3673305.0424799253]

In [300]:
[np.sum(solver_stage_5b.get_state(x, model)[0,:]) for x in shist_s5b], np.sum(initial_conditions)

([291378.52234196156,
  335394.5003458088,
  341074.4971532446,
  351205.47087369615,
  371330.4896747055,
  363381.82147233846,
  331081.3825934498,
  388908.0018470183,
  349464.27367570176,
  376175.9617197926],
 200010)

In [301]:
reff_fn = ca.Function('reff_fn', [solver.decision_vars], [model.xs[0,0]/ca.sum2(model.xs[0,:]) * model.ps[0] / (model.ps[2] + model.ps[4] + model.ps[-2])])

In [302]:
[reff_fn(x['x']) for x in shist_s5b]

[DM(1.57267),
 DM(1.23522),
 DM(1.19111),
 DM(1.0922),
 DM(1.05449),
 DM(1.03524),
 DM(1.03969),
 DM(1.16609),
 DM(1.24958),
 DM(1.18386)]

In [303]:
initial_conditions[0]/sum(initial_conditions) * true_parameters[0] / (true_parameters[2] + true_parameters[4] + true_parameters[-2])

1.9606862794115199

In [383]:
gamma_profile_config_5b = {
    'g+': model.ps[1],
    'pidx': ca.Function('pidx_mort', [solver_stage_5b.decision_vars], [model.ps[1]])
}
gamma_profiler_5b = pypei.fitter.Profiler(solver_stage_5b, gamma_profile_config_5b)
solver_stage_5b.profilers = [gamma_profiler_5b]

In [384]:
mle_gamma = float(gamma_profiler.p_locator(shist_s5b[6]['x']))
print(mle_gamma)
gammapbounds = [np.linspace(mle_gamma, 0, 11), np.linspace(mle_gamma, 2*mle_gamma, 11)]

400.35042255912765


In [385]:
gamma_profiles = solver_stage_5b.profile(shist_s5b[6], p=p_5a, w0=whist_s5b[6], nit=1,
                                        weight=huber_5a, y=data_5b, hist=False, repair=True,
                                        lbx=lbx, lbg=0, weight_args=weight_args_stage_5b,
                                        pbounds=[gammapbounds])

Iteration: 0
This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        1
Number of nonzeros in inequality constraint Jacobian.:     6822
Number of nonzeros in Lagrangian Hessian.............:     7696

Total number of variables............................:      367
                     variables with only lower bounds:        7
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:     1800
        inequality constraints with only lower bounds:     1800
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_

In [386]:
armort_ll = np.array([float(x['s']['f']) for x in gamma_profiles[0].values()])
norm_mort_prof = np.exp(-0.5*(armort_ll - min(armort_ll)))
plt.figure()
plt.plot(gamma_profiles[0].keys(), norm_mort_prof, 'ro')
# plt.plot(np.concatenate(mortprofiles['ps']), norm_mort_prof, 'ko')
# plt.title('Profile of Deaths')
# plt.xlabel('Total Deaths')
plt.ylabel('Objective Function Value')
plt.axvline(mle_gamma)
plt.axvline(true_parameters[1], color='pink')
plt.axhline(0.14650006448608432)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Stage 6: with sum(x(0))

In [751]:
objective_config_stage_6 = {
    'Y': [
        {
            'sz': data_5b.shape,
            'obs_fn': comparator5b,
        },
        {
            'sz': model.xs.shape,
            'unitary': True,
            'obs_fn': pypei.objective.Objective._MODELFIT(model),
        },
        # "initial unvaccinated proportion" [regularised towards truth]
        {
            'sz': (1, 1),
            'obs_fn': model.xs[0,0] / ca.sum2(model.xs[0,:]),
        },
        # "regularisation of gamma (rate of latent -> infectious)" [regularised towards 0]
        {
            'sz': (1, 1),
            'obs_fn': model.ps[1],
        },
    ],
    'L': [
        data_L_5b,
        mdl_L,
        {'n': 1, 'iden': True, 'w': 1000},
        {'n': 1, 'iid': True},
    ]
}
objective_stage_6 = pypei.objective.Objective()
objective_stage_6.make(objective_config_stage_6)

In [752]:
solver_stage_6 = pypei.irls_fitter.Solver(objective=objective_stage_6)
solver_config_stage_6 = solver_stage_6.make_config(model, objective_stage_6)
solver_config_stage_6['o'] = {'ipopt': {'max_iter': 5000, 'print_frequency_iter': 100}}
solver_stage_6.make(solver_config_stage_6)

In [753]:
def p_6(w, y_noisy):
    return [*w, 6, *[i for l in y_noisy for i in l], 0, initial_conditions[0]/sum(initial_conditions), 0]

weight_args_stage_6 = {'structure': (data_L_5b['struct'], mdl_L['struct']), 'bounds': [[5e-2, None]]*5 + [[None, 3]]*5}

In [754]:
solver_stage_6_output = solver_stage_6.irls(x0x0, p_6, y=data_5b, nit=10, lbx=lbx, lbg=0,
                                    w0=w0_s5a, weight=huber_5a, weight_args=weight_args_stage_6, 
                                    hist=True)
sol_s6, ws_s6, shist_s6, whist_s6, rshist_s6, errhist_s6 = solver_stage_6_output

Iteration: 0
This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:     6822
Number of nonzeros in Lagrangian Hessian.............:     7713

Total number of variables............................:      367
                     variables with only lower bounds:        7
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:     1800
        inequality constraints with only lower bounds:     1800
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_

/media/dwu402/Data/cashpack/pypei/irls_fitter.py:231: StepControlWarning: Log-Likelihood is not improving: 0.0002611556755748927
  


 200  2.6592759e+02 0.00e+00 2.57e+03  -1.0 2.36e+00  -4.9 1.00e+00 1.00e+00f  1
 300  2.7856500e+02 0.00e+00 2.27e+03  -1.0 5.72e-01  -2.4 1.00e+00 1.00e+00f  1
 400  2.8564009e+02 0.00e+00 5.81e+03  -1.0 2.21e+01  -5.9 1.00e+00 1.00e+00f  1
 500  2.9800719e+02 0.00e+00 5.10e+02  -1.0 1.10e+00  -3.9 1.00e+00 1.00e+00f  1
 600  2.9955405e+02 0.00e+00 4.55e+01  -1.0 2.46e-01  -2.4 1.00e+00 1.00e+00f  1
 700  3.3700612e+02 0.00e+00 5.72e+02  -1.0 1.59e+01  -5.8 1.00e+00 1.00e+00f  1
 800  3.5539139e+02 0.00e+00 1.52e+04  -1.0 2.87e+01  -6.1 1.00e+00 1.00e+00f  1
 900  3.7959859e+02 0.00e+00 2.35e+04  -1.0 1.72e+02  -6.4 1.00e+00 2.50e-01f  3
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1000  2.4494928e+02 0.00e+00 2.99e+03  -1.7 8.80e+01  -6.2 1.00e+00 6.25e-02f  5
1100  2.2967925e+02 0.00e+00 9.18e+03  -1.7 1.19e+02  -7.0 1.00e+00 2.50e-01f  3
1200  2.1955441e+02 0.00e+00 1.55e+04  -1.7 3.99e+01  -6.8 1.00e+00 1.00e+00f  1
1300  2.1201147e+02 0.00e+00

/media/dwu402/Data/cashpack/pypei/irls_fitter.py:231: StepControlWarning: Log-Likelihood is not improving: 0.00030109027920441887
  


In [755]:
get_cs = ca.Function('get_cs', [solver.decision_vars], model.cs)
plt.figure()
plt.plot(data_time_max_qual, data_max_qual[-2,:].T, 'k--')

finvals = []
for sh5b in shist_s6:
    compare = model.all_x_at(get_cs(sh5b['x']), data_time_max_qual)[:,-2]
    finvals.append(float(compare[-1,0]))
    plt.plot(data_time_max_qual, compare)
plt.legend(['true'] + list(range(len(shist_s5b))))

plt.plot(data_time_partial_window, data_5b.T, '.')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [756]:
plt.figure()
plt.plot(finvals)
plt.axhline(data_max_qual[-2,-1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [757]:
plt.figure()
plt.plot(model.observation_times, solver_stage_6.get_state(shist_s6[7], model))
plt.gca().set_prop_cycle(None)
plt.plot(data_time_max_qual, data_max_qual.T, '.', mfc='none')
plt.plot(data_time_partial_window, data_5b.T, 'x')
# plt.ylim(0, 30000)
plt.legend(['S','E','I','H','G','R','D','cE','cH'])
plt.title("Fit to im-perfect, noisy partial data [stage 6] with IRLS-GP")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Fit to im-perfect, noisy partial data [stage 6] with IRLS-GP')

In [489]:
plt.figure()
plt.semilogy(whist_s6)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [490]:
prop = ca.Function('prop_S_fn', [solver.decision_vars, objective_stage_6.y0s[2]], [objective_stage_6.obj_fn(2)])

In [491]:
[float(prop(x['x'], initial_conditions[0]/sum(initial_conditions))) for x in shist_s6]

[7.387467596164691e-07,
 2.1750287544804902e-07,
 139.63051012040953,
 82.34958360325267,
 0.0008526717202506646,
 0.0008387864690865786,
 0.0008311587690675797,
 0.0008300462947197462,
 0.0008298877690396523,
 0.0008298654388186763]

In [492]:
plt.figure()
plt.semilogy([np.squeeze(solver.get_parameters(s, model)) for s in shist_s6])
plt.gca().set_prop_cycle(None)
for i, p_by_i in enumerate(true_parameters):
    plt.semilogy(range(len(shist_s6)), [p_by_i]*len(shist_s6), 'o', alpha=0.7, mfc='None')
plt.title("Parameters by iteration")
plt.legend(list("bgedam") + ['muH', 'Truth'])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [493]:
gamma_profile_config = {
    'g+': model.ps[1],
    'pidx': ca.Function('pidx_mort', [solver_stage_6.decision_vars], [model.ps[1]])
}
gamma_profiler = pypei.fitter.Profiler(solver_stage_6, gamma_profile_config)
solver_stage_6.profilers = [gamma_profiler]

In [494]:
mle_gamma = float(gamma_profiler.p_locator(shist_s6[7]['x']))
print(mle_gamma)
gammapbounds = [np.linspace(mle_gamma, 0, 11), np.linspace(mle_gamma, 2*mle_gamma, 11)]

0.5428560344971044


In [534]:
p_reg_fn = ca.Function("pregfn", [solver_stage_6.decision_vars, solver_stage_6.parameters], [objective_stage_6.us_obj_comp(3)])

p_reg_fn(shist_s6[7]['x'], p_6(whist_s6[7], data_5b))

DM(-0.542856)

In [505]:
def p_6_reg(reg_w):
    def p_6(w, y_noisy):
        return [*w, reg_w, *[i for l in y_noisy for i in l], 0, initial_conditions[0]/sum(initial_conditions), 0]
    return p_6


In [ ]:
gamma_profiles_collated = []
gamma_lin = range(0, 27, 2)
for reg_str in gamma_lin:
    
    gamma_profiles = solver_stage_6.profile(shist_s6[7], p=p_6_reg(reg_str), w0=whist_s6[7], nit=1,
                                            weight=huber_5a, y=data_5b, hist=False, repair=True,
                                            lbx=lbx, lbg=0, weight_args=weight_args_stage_6,
                                            pbounds=[gammapbounds])

    gamma_profiles_collated.append(gamma_profiles)
    

Iteration: 0
This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        1
Number of nonzeros in inequality constraint Jacobian.:     6822
Number of nonzeros in Lagrangian Hessian.............:     7713

Total number of variables............................:      367
                     variables with only lower bounds:        7
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:     1800
        inequality constraints with only lower bounds:     1800
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_

In [781]:
for reg_str, gamma_profiles in zip(gamma_lin, gamma_profiles_collated):
    armort_ll = np.array([float(x['s']['f']) for x in gamma_profiles[0].values()])
    norm_mort_prof = np.exp(-0.5*(armort_ll - min(armort_ll)))

    plt.figure()
    plt.plot(gamma_profiles[0].keys(), norm_mort_prof, 'ro', label='Profile')
    # plt.plot(np.concatenate(mortprofiles['ps']), norm_mort_prof, 'ko')
    # plt.title('Profile of Deaths')
    plt.xlabel('$\\gamma$', fontproperties=label_font)
    plt.ylabel('Normalised Likelihood Value', fontproperties=label_font)
    plt.xticks(fontproperties=tick_font)
    plt.yticks(fontproperties=tick_font)
#     plt.axvline(mle_gamma)
    plt.axvline(true_parameters[1], color='dodgerblue', linestyle='dashed', label='Truth')
    plt.axhline(0.14650006448608432, color='dodgerblue', label='95% CI Threshold')
    plt.title(f"$\\rho = {reg_str}$", fontproperties=label_font)
    plt.legend(prop=legend_font)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [788]:
plt.figure(232)
plt.savefig('/home/dwu402/REPOS/writing/samoa/img/synthetic_rho_24.pdf', dpi=300)

In [ ]:
def concavity(f1, f2, f3):
    return f1/2 - f2 + f3/2

def detect_anomaly(ll_seq):
    # cannot do endpoints
    return [True] + [float(concavity(f1, f2, f3)) > 0 for f1, f2, f3 in zip(ll_seq[:-2], ll_seq[1:-1], ll_seq[2:])] + [True]

def filter_by(seq, cond):
    return [elem for elem, check in zip(seq, cond) if bool(check)]

In [801]:
# plt.figure()
max_locs = []
for gamma_profiles in gamma_profiles_collated:
    armort_ll = np.array([float(x['s']['f']) for x in gamma_profiles[0].values()])
    norm_mort_prof = np.exp(-0.5*(armort_ll - min(armort_ll)))
    
    anomalies = detect_anomaly(armort_ll)
    
    profile_x = filter_by(gamma_profiles[0].keys(), anomalies)
    profile_y = filter_by(norm_mort_prof, anomalies)
    
    spl = interpolate.UnivariateSpline(profile_x, profile_y, k=3, s=1e-6)
#     ln, = plt.plot(profile_x, profile_y, 'o')
#     plt.plot(np.linspace(0,1.1,1001), spl(np.linspace(0,1.1,1001)), color=ln.get_color())
    grid = np.linspace(0, 1.1, 1001)
    max_locs.append(grid[np.argmax(spl(grid))])
#     max_locs.append(
plt.figure()
plt.plot(list(gamma_lin), max_locs)
plt.axhline(true_parameters[1], color='red', linestyle='--', label='Truth')
plt.title("Estimated Parameter Value of $\\gamma$ at Maximum Likelihood Estimate", fontproperties=legend_font)
plt.xlabel("Regularistion strength, $\\rho$", fontproperties=label_font)
plt.xticks(fontproperties=tick_font)
plt.ylabel("$\\gamma$", fontproperties=label_font)
plt.yticks(fontproperties=tick_font)
plt.legend(prop=legend_font)
plt.savefig('/home/dwu402/REPOS/writing/samoa/img/synthetic_rho_gamma.pdf', dpi=300)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [784]:
gloc = interpolate.interp1d(gamma_lin, max_locs)
optimize.fsolve(lambda x: gloc(x) - true_parameters[1], 16)

array([17.41818182])

In [637]:
gamma_reg_component = ca.Function('gamma_reg_comp', [solver_stage_6.decision_vars, solver_stage_6.parameters], [objective_stage_6.us_obj_fn(-1)])
fit_components = ca.Function('everything_else', [solver_stage_6.decision_vars, solver_stage_6.parameters], [sum(objective_stage_6.obj_fn(i) for i in range(3))])

In [715]:
gregs = []
ofits = []

for i, gamma_profile in enumerate(gamma_profiles_collated):
    armort_ll = np.array([float(x['s']['f']) for x in gamma_profile[0].values()])
    aidx = np.argmin(armort_ll)
    greg = gamma_reg_component(gamma_profile[0].peekitem(aidx)[1]['s']['x'], p_6_reg(i)(whist_s6[7], data_5b))
    ofit = fit_components(gamma_profile[0].peekitem(aidx)[1]['s']['x'], p_6_reg(i)(whist_s6[7], data_5b))
    
    gregs.append(float(greg))
    ofits.append(float(ofit))

In [774]:
plt.figure()
plt.plot(gregs, ofits, 'x-')
plt.plot(gregs[8], ofits[8], 'ro')
plt.title("Tradeoff curve for gamma regularisation")
plt.xlabel("gamma regualrisation component (unscaled)")
plt.ylabel("other components (scaled)")
# plt.xscale('log')
# plt.yscale('log')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'other components (scaled)')

In [777]:
get_cs = ca.Function('get_cs', [solver_stage_6.decision_vars], model.cs)

def validation_error(solution):
    xsol = solution['x']
    x_perf = model.all_x_at(get_cs(xsol), data_time_max_qual).T
    return np.linalg.norm(x_perf - data_max_qual, ord=1)

def get_mlsol_from_profile(profile):
    armort_ll = np.array([float(x['s']['f']) for x in profile[0].values()])
    aidx = np.argmin(armort_ll)
    return profile[0].peekitem(aidx)[1]['s']

verrs_ml_porfile_gamma_reg = [validation_error(get_mlsol_from_profile(gamma_profile)) for gamma_profile in gamma_profiles_collated]

In [778]:
plt.figure()
plt.semilogx(gregs, verrs_ml_porfile_gamma_reg, 'x-')
plt.semilogx(gregs[8], verrs_ml_porfile_gamma_reg[8], 'rx')
plt.xlabel('gamma reg. component')
plt.ylabel('validation error')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'validation error')

In [803]:
plt.figure()
plt.plot(gamma_lin, verrs_ml_porfile_gamma_reg)
plt.xlabel('Regularisation Strength, $\\rho$', fontproperties=label_font)
plt.ylabel('Validation Error', fontproperties=label_font)
plt.xticks(fontproperties=tick_font)
plt.yticks(fontproperties=tick_font)
plt.savefig("/home/dwu402/REPOS/writing/samoa/img/synthetic_validation_gamma.pdf", dpi=300)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [726]:
model.xs.shape

(200, 9)

In [730]:
t_pop = ca.Function('t_pop', [solver.decision_vars], [ca.sum2(model.xs[0,:])])

t_pop_mlprof = [float(t_pop(get_mlsol_from_profile(profile)['x'])) for profile in gamma_profiles_collated]

In [733]:
plt.figure()
plt.plot(list(gamma_lin), t_pop_mlprof)
plt.axhline(sum(initial_conditions), color='purple')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [696]:
gamma_profiles_collated_logger = []
greg_logspace = np.logspace(0, 2, num=11, base=10)
for reg_str in greg_logspace:
    
    gamma_profiles = solver_stage_6.profile(shist_s6[7], p=p_6_reg(reg_str), w0=whist_s6[7], nit=1,
                                            weight=huber_5a, y=data_5b, hist=False, repair=True,
                                            lbx=lbx, lbg=0, weight_args=weight_args_stage_6,
                                            pbounds=[gammapbounds])

    gamma_profiles_collated_logger.append(gamma_profiles)

Iteration: 0
This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        1
Number of nonzeros in inequality constraint Jacobian.:     6822
Number of nonzeros in Lagrangian Hessian.............:     7713

Total number of variables............................:      367
                     variables with only lower bounds:        7
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:     1800
        inequality constraints with only lower bounds:     1800
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_

In [697]:
gregs_l = []
ofits_l = []

for i, gamma_profile in enumerate(gamma_profiles_collated_logger):
    armort_ll = np.array([float(x['s']['f']) for x in gamma_profile[0].values()])
    aidx = np.argmin(armort_ll)
    greg = gamma_reg_component(gamma_profile[0].peekitem(aidx)[1]['s']['x'], p_6_reg(greg_logspace[i])(whist_s6[7], data_5b))
    ofit = fit_components(gamma_profile[0].peekitem(aidx)[1]['s']['x'], p_6_reg(greg_logspace[i])(whist_s6[7], data_5b))
    
    gregs_l.append(float(greg))
    ofits_l.append(float(ofit))

In [705]:
plt.figure()
plt.plot(gregs_l, ofits_l, 'x-')
plt.plot(gregs_l[0], ofits_l[0], 'ro')
plt.title("Tradeoff curve for gamma regularisation")
plt.xlabel("gamma regualrisation component (unscaled)")
plt.ylabel("other components (scaled)")
# plt.xscale('log')
# plt.yscale('log')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'other components (scaled)')

In [701]:
verrs_ml_porfile_gamma_reg_log = [validation_error(get_mlsol_from_profile(gamma_profile)) for gamma_profile in gamma_profiles_collated_logger]

plt.figure()
plt.plot(gregs_l, verrs_ml_porfile_gamma_reg_log, 'x-')
plt.ylabel('validation error')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'validation error')

In [707]:
greg_logspace[np.argmin(verrs_ml_porfile_gamma_reg_log)]

15.848931924611142

In [708]:
for gamma_profiles in gamma_profiles_collated_logger:
    armort_ll = np.array([float(x['s']['f']) for x in gamma_profiles[0].values()])
    norm_mort_prof = np.exp(-0.5*(armort_ll - min(armort_ll)))

    plt.figure()
    plt.plot(gamma_profiles[0].keys(), norm_mort_prof, 'ro')
    # plt.plot(np.concatenate(mortprofiles['ps']), norm_mort_prof, 'ko')
    # plt.title('Profile of Deaths')
    # plt.xlabel('Total Deaths')
    plt.ylabel('Objective Function Value')
    plt.axvline(mle_gamma)
    plt.axvline(true_parameters[1], color='pink')
    plt.axhline(0.14650006448608432)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [674]:
# [solver_stage_6.get_state(x['s'], model) for x in gamma_profiles[0].values()]
           
plt.figure()
for x in gamma_profiles[0].values():
    plt.plot(model.observation_times, solver_stage_6.get_state(x['s'], model))
    plt.gca().set_prop_cycle(None)
plt.plot(data_time_max_qual, data_max_qual.T, '.', mfc='none')
plt.plot(data_time_partial_window, data_5b.T, 'x')
# plt.ylim(0, 30000)
plt.legend(['S','E','I','H','G','R','D','cE','cH'], loc='upper left', bbox_to_anchor=[1.01, 1])
plt.title("Fit to im-perfect, noisy partial data [stage 6] with IRLS-GP")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Fit to im-perfect, noisy partial data [stage 6] with IRLS-GP')

# Stage 6a: no gamma reg.

In [456]:
objective_config_stage_6a = {
    'Y': [
        {
            'sz': data_5b.shape,
            'obs_fn': comparator5b,
        },
        {
            'sz': model.xs.shape,
            'unitary': True,
            'obs_fn': pypei.objective.Objective._MODELFIT(model),
        },
        {
            'sz': (1, 1),
            'obs_fn': model.xs[0,0] / ca.sum2(model.xs[0,:]),
        },
    ],
    'L': [
        data_L_5b,
        mdl_L,
        {'n': 1, 'iden': True, 'w': 1000},
    ]
}
objective_stage_6a = pypei.objective.Objective()
objective_stage_6a.make(objective_config_stage_6a)

In [457]:
solver_stage_6a = pypei.irls_fitter.Solver(objective=objective_stage_6a)
solver_config_stage_6a = solver_stage_6a.make_config(model, objective_stage_6a)
solver_config_stage_6a['o'] = {'ipopt': {'max_iter': 5000, 'print_frequency_iter': 100}}
solver_stage_6a.make(solver_config_stage_6a)

In [458]:
def p_6a(w, y_noisy):
    return [*w, *[i for l in y_noisy for i in l], 0, initial_conditions[0]/sum(initial_conditions)]

weight_args_stage_6a = {'structure': (data_L_5b['struct'], mdl_L['struct']), 'bounds': [[5e-2, None]]*5 + [[None, 3]]*5}

In [459]:
solver_stage_6a_output = solver_stage_6a.irls(x0x0, p_6a, y=data_5b, nit=10, lbx=lbx, lbg=0,
                                    w0=w0_s5a, weight=huber_5a, weight_args=weight_args_stage_6a, 
                                    hist=True)
sol_s6a, ws_s6a, shist_s6a, whist_s6a, rshist_s6a, errhist_s6a = solver_stage_6a_output

Iteration: 0
This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:     6822
Number of nonzeros in Lagrangian Hessian.............:     7713

Total number of variables............................:      367
                     variables with only lower bounds:        7
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:     1800
        inequality constraints with only lower bounds:     1800
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_

/media/dwu402/Data/cashpack/pypei/irls_fitter.py:231: StepControlWarning: Log-Likelihood is not improving: 1.0012816644982268
  warnings.warn(f"Log-Likelihood is not improving: {err}", category=Solver.StepControlWarning)


 200  2.8630688e+02 0.00e+00 3.06e+01  -5.7 7.21e+03 -12.8 1.00e+00 1.00e+00f  1
 300  1.1023626e+02 0.00e+00 1.85e+00  -5.7 4.45e+02 -11.8 1.00e+00 1.00e+00h  1
 400  6.8858941e+01 0.00e+00 9.30e-02  -5.7 5.64e+02 -12.5 1.00e+00 1.00e+00h  1
 500  5.4686006e+01 0.00e+00 5.91e-01  -5.7 4.59e+02 -13.6 1.00e+00 1.00e+00h  1
 600  4.9200456e+01 0.00e+00 1.69e-02  -5.7 6.82e+00 -10.0 1.00e+00 1.00e+00h  1
 700  2.0609962e+03 0.00e+00 1.19e+01  -8.6 1.17e+02 -10.8 1.00e+00 1.00e+00H  1
 800  3.9339174e+01 0.00e+00 1.25e-01  -8.6 2.59e+04    -  2.70e-01 1.09e-02h  5
 900  3.8121790e+01 0.00e+00 9.41e-02  -9.0 1.57e+04    -  1.00e+00 3.21e-02h  5
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1000  3.9062294e+01 0.00e+00 5.95e-06  -9.0 1.11e-01  -4.7 1.00e+00 1.00e+00F  1

Number of Iterations....: 1079

                                   (scaled)                 (unscaled)
Objective...............:   6.8284266932632934e-03    3.7426345843187050e+01
Dual infe

In [461]:
get_cs = ca.Function('get_cs', [solver.decision_vars], model.cs)
plt.figure()
plt.plot(data_time_max_qual, data_max_qual[-2,:].T, 'k--')

finvals = []
for sh in shist_s6a:
    compare = model.all_x_at(get_cs(sh['x']), data_time_max_qual)[:,-2]
    finvals.append(float(compare[-1,0]))
    plt.plot(data_time_max_qual, compare)
plt.legend(['true'] + list(range(len(shist_s6a))))

plt.plot(data_time_partial_window, data_5b.T, '.')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [462]:
plt.figure()
plt.semilogy([np.squeeze(solver.get_parameters(s, model)) for s in shist_s6a])
plt.gca().set_prop_cycle(None)
for i, p_by_i in enumerate(true_parameters):
    plt.semilogy(range(len(shist_s6a)), [p_by_i]*len(shist_s6a), 'o', alpha=0.7, mfc='None')
plt.title("Parameters by iteration")
plt.legend(list("bgedam") + ['muH', 'Truth'])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [463]:
gamma_profile_config6a = {
    'g+': model.ps[1],
    'pidx': ca.Function('pidx_mort', [solver_stage_6a.decision_vars], [model.ps[1]])
}
gamma_profiler6a = pypei.fitter.Profiler(solver_stage_6a, gamma_profile_config6a)
solver_stage_6a.profilers = [gamma_profiler6a]

In [467]:
mle_gamma_6a = float(gamma_profiler6a.p_locator(shist_s6a[7]['x']))
print(mle_gamma_6a)
gammapbounds_6a = [np.linspace(mle_gamma_6a, 0, 11), np.linspace(mle_gamma_6a, 2*mle_gamma_6a, 11)]
gammapbounds_6a

25.48903085283048


[array([25.48903085, 22.94012777, 20.39122468, 17.8423216 , 15.29341851,
        12.74451543, 10.19561234,  7.64670926,  5.09780617,  2.54890309,
         0.        ]),
 array([25.48903085, 28.03793394, 30.58683702, 33.13574011, 35.68464319,
        38.23354628, 40.78244936, 43.33135245, 45.88025554, 48.42915862,
        50.97806171])]

In [468]:
gamma_profiles = solver_stage_6a.profile(shist_s6a[7], p=p_6a, w0=whist_s6a[6], nit=1,
                                        weight=huber_5a, y=data_5b, hist=False, repair=True,
                                        lbx=lbx, lbg=0, weight_args=weight_args_stage_6a,
                                        pbounds=[gammapbounds])

Iteration: 0
This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        1
Number of nonzeros in inequality constraint Jacobian.:     6822
Number of nonzeros in Lagrangian Hessian.............:     7713

Total number of variables............................:      367
                     variables with only lower bounds:        7
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:     1800
        inequality constraints with only lower bounds:     1800
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_

In [471]:
armort_ll_6a = np.array([float(x['s']['f']) for x in gamma_profiles_6a[0].values()])
norm_mort_prof_6a = np.exp(-0.5*(armort_ll_6a - min(armort_ll_6a)))
plt.figure()
plt.plot(gamma_profiles_6a[0].keys(), norm_mort_prof_6a, 'ro')
# plt.plot(np.concatenate(mortprofiles['ps']), norm_mort_prof, 'ko')
# plt.title('Profile of Deaths')
# plt.xlabel('Total Deaths')
plt.ylabel('Objective Function Value')
plt.axvline(mle_gamma)
plt.axvline(true_parameters[1], color='pink')
plt.axhline(0.14650006448608432)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …